<a href="https://colab.research.google.com/github/TomGermeau/BlancPain/blob/main/Project_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install and update spaCy
!pip install -U spacy

# Download the english language model
!python -m spacy download fr_core_news_sm


     |████████████████████████████████| 6.0 MB 23.3 MB/s 
     |████████████████████████████████| 181 kB 48.0 MB/s 
     |████████████████████████████████| 628 kB 34.3 MB/s 
     |████████████████████████████████| 42 kB 991 kB/s 
     |████████████████████████████████| 451 kB 49.8 MB/s 
     |████████████████████████████████| 10.1 MB 38.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 17.4 

In [32]:
# Import required packages
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/TomGermeau/BlancPain/main/training_data.csv")
df.head()


,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4800 non-null   int64 
 1   sentence    4800 non-null   object
 2   difficulty  4800 non-null   object
dtypes: int64(1), object(2)
memory usage: 112.6+ KB


In [9]:
df.difficulty.value_counts()

A1    813
C2    807
C1    798
B1    795
A2    795
B2    792
Name: difficulty, dtype: int64

In [12]:
# Create a list of punctuation marks
punctuations = string.punctuation

punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
stop_words = fr_stop

list(stop_words)[:10]

['avait',
 'directement',
 'dix-neuf',
 'hormis',
 'divers',
 'comme',
 'siennes',
 'permet',
 'ni',
 'serait']

In [52]:
sp = spacy.load('fr_core_news_sm')

# Create tokenizer function
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    ## alternative way
    # mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

# Example
sentence = df["sentence"].sample()




'Il est également connu pour ses publicités télévisées à faible budget et ses affiches imprimées à Albuquerque, dans lesquels il s\'annonce principalement sous le slogan : "Appelez donc Saul !"'

['portable',
 'permettre',
 'jeune',
 'garder',
 'espace',
 'priver',
 'sein',
 'vie',
 'familial']

'Les bioessais effectués ont montré que les mêmes symptômes de morbidité sont observés chez les deux espèces et avec les deux métaux.'

In [24]:
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer)

In [7]:
X = df['sentence'] # the features we want to analyze
ylabels = df['difficulty'] # the labels, or answers, we want to test against

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=1234, stratify=ylabels)

X_train

962     Le réalisateur m'a d'abord demandé de me mettr...
1886    Après quelques mois de cette pauvreté noble, a...
2721    L'indicateur n'était que de 40% chez les femme...
1025    L'objectif de ce type de voyage est d'être act...
4048    Et, en France, beaucoup moins de filles que de...
                              ...                        
3693     Je vais prendre ma douche dans ma salle-de-bain.
3408    Après l'éruption de 1754, la plus grosse connu...
4289    Léonard est initié par Verrocchio aux nombreus...
3312    On en trouve des exemples dans l'ouvrage "L'in...
269     On peut aussi aller au théâtre, dans les musée...
Name: sentence, Length: 3840, dtype: object

In [25]:
y_train

962     B1
1886    C1
2721    A2
1025    B1
4048    B2
        ..
3693    A1
3408    B1
4289    C2
3312    C2
269     A1
Name: difficulty, Length: 3840, dtype: object

In [40]:
def evaluate(test, pred):
    precision = precision_score(test, pred, 
                                           pos_label='positive',
                                           average='micro')
    recall = recall_score(test, pred, 
                                           pos_label='positive',
                                           average='micro')
    f1 = f1_score(test, pred, average='micro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(test, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(test, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [49]:
# Use random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
# Define vectorizer
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer

# Define classifier

#classifier = RandomForestClassifier(n_estimators=1000, oob_score = 'TRUE')
#classifier =svm.SVC(decision_function_shape='ovo')
# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])


# Fit model on training set
pipe.fit(X_train, y_train)

# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
evaluate(y_test, y_pred)

# Evaluation - training set
evaluate(y_train, pipe.predict(X_train))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1365: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1365: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,


CONFUSION MATRIX:
[[74 32 26 17  8  6]
 [28 58 35 16 14  8]
 [15 29 50 31 11 23]
 [ 5  5 14 64 35 35]
 [ 2  7  6 33 80 32]
 [ 3  6 15 24 33 80]]
ACCURACY SCORE:
0.4229
CLASSIFICATION REPORT:
	Precision: 0.4229
	Recall: 0.4229
	F1_Score: 0.4229
CONFUSION MATRIX:
[[616  22   9   2   0   1]
 [ 16 612   6   0   2   0]
 [  9   8 618   1   0   0]
 [  0   0   2 628   1   3]
 [  0   0   0   0 638   0]
 [  0   0   0   0   2 644]]
ACCURACY SCORE:
0.9781
CLASSIFICATION REPORT:
	Precision: 0.9781
	Recall: 0.9781
	F1_Score: 0.9781


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1365: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1365: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  UserWarning,
